In [ ]:
"""
Purpose: To look at the initial stats of the 
graph from autoproofreading


"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/meshAfterParty")
sys.path.append("/meshAfterParty/meshAfterParty/graph/")

In [3]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-28 19:01:59,301 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-28 19:01:59,303 - settings - Setting database.user to celiib
INFO - 2021-02-28 19:01:59,304 - settings - Setting database.password to newceliipass
INFO - 2021-02-28 19:01:59,308 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-28 19:01:59,309 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-28 19:01:59,324 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-28 19:01:59,685 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-28 19:01:59,689 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-28 19:01:59,690 - settings - Setting database.user to celiib
INFO - 2021-02-28 19:01:59,691 - settings - Setting database.password to newceliipass
INFO - 2021-02-28 19:01:59,692 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-28 19:01:59,692 - settings - Setting database.user to celiib
INFO - 2021-02-28 19:01:59,693 - settings - Setting database.password to newceliipass
INFO - 2021-02-28 19:01:59,694 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-28 19:02:00,109 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-28 19:02:00,140 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-28 19:02:00,141 - settings - Setting database.user to celiib
INFO - 2021-02-28 19:02:00,141 - settings - Setting database.pa

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu

# Checking the Status of Version 3

In [28]:
unique_errors,errors_counts = np.unique(((schema.jobs & "table_name='__auto_proofread_neurons3'") & "status='error'").fetch("error_message"),return_counts=True)

In [30]:
errors_counts[errors_counts>5]

array([  7,   6, 112, 146,  10,  33,  80,  70, 334])

In [29]:
unique_errors[errors_counts>5]

array(['Exception: The number of nodes in the concept graph and number of branches passed to it did not match\nlen(curr_limb_concept_network.nodes())=3, len(curr_limb_divided_skeletons)= 4',
       'Exception: The number of nodes in the concept graph and number of branches passed to it did not match\nlen(curr_limb_concept_network.nodes())=3, len(curr_limb_divided_skeletons)= 6',
       'Exception: not downsampled branch',
       'IndexError: index 1 is out of bounds for axis 0 with size 1',
       'IndexError: index 2 is out of bounds for axis 0 with size 1',
       'IndexError: index 2 is out of bounds for axis 0 with size 2',
       'IndexError: list index out of range',
       'IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices',
       'SystemExit: SIGTERM received'], dtype=object)

In [38]:
(schema.jobs & "table_name='__auto_proofread_neurons3'" & "error_message='Exception: not downsampled branch'")

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__auto_proofread_neurons3,02397fe41858f71f4c88c200fa48ced2,error,=BLOB=,Exception: not downsampled branch,=BLOB=,celiib@10.28.0.154,at-node27,1,183318,2021-02-28 00:03:48
__auto_proofread_neurons3,04dafb2b982cd9a6497d6e36abbe89b6,error,=BLOB=,Exception: not downsampled branch,=BLOB=,celiib@10.28.0.147,at-node20,1,184313,2021-02-28 00:03:49
__auto_proofread_neurons3,05c0b68408ae8ab6d4f533a02ffa7e65,error,=BLOB=,Exception: not downsampled branch,=BLOB=,celiib@10.28.0.83,at-compute004,1,185003,2021-02-28 04:43:32
__auto_proofread_neurons3,089f9dc8bf720c32034e36f10ad9e9bf,error,=BLOB=,Exception: not downsampled branch,=BLOB=,celiib@10.28.0.144,at-node17,1,183964,2021-02-27 23:07:20
__auto_proofread_neurons3,11254f7f61e46b4c79c18214929c34e0,error,=BLOB=,Exception: not downsampled branch,=BLOB=,celiib@10.28.0.136,at-node9,1,184191,2021-02-28 08:17:41
__auto_proofread_neurons3,1141790ca035ad29e985cba29cae7b23,error,=BLOB=,Exception: not downsampled branch,=BLOB=,celiib@10.66.4.27,at-compute005,1,183188,2021-02-28 04:29:08
__auto_proofread_neurons3,1177a2340a26d33c933dcccb7a136638,error,=BLOB=,Exception: not downsampled branch,=BLOB=,celiib@10.28.0.144,at-node17,1,185013,2021-02-28 08:17:40
__auto_proofread_neurons3,12c21b3582e17e4c5d1be643d3b3ef14,error,=BLOB=,Exception: not downsampled branch,=BLOB=,celiib@10.28.0.165,at-node38,1,185103,2021-02-28 08:17:41
__auto_proofread_neurons3,17b97a93e5b4a9716ff7a2a7968050b9,error,=BLOB=,Exception: not downsampled branch,=BLOB=,celiib@10.28.0.83,at-compute004,1,183875,2021-02-27 23:07:21
__auto_proofread_neurons3,18995558b6fca3a8e0262ea87a31e727,error,=BLOB=,Exception: not downsampled branch,=BLOB=,celiib@10.66.4.27,at-compute005,1,183207,2021-02-28 04:29:08


In [40]:
print((schema.jobs & "table_name='__auto_proofread_neurons3'" & dict(key_hash="11254f7f61e46b4c79c18214929c34e0")).fetch1("error_stack"))

Traceback (most recent call last):
  File "/meshAfterParty/neuron_utils.py", line 719, in branches_to_concept_network
    curr_branch_meshes_downsampled.append(sk.resize_skeleton_branch(b,n_segments=1))
  File "/meshAfterParty/skeleton_utils.py", line 3799, in resize_skeleton_branch
    cleaned_skeleton = convert_branch_graph_to_skeleton(skeleton_graph)
  File "/meshAfterParty/skeleton_utils.py", line 3756, in convert_branch_graph_to_skeleton
    skeleton_node_coordinates = get_ordered_branch_nodes_coordinates(skeleton_graph)
  File "/meshAfterParty/skeleton_utils.py", line 3568, in get_ordered_branch_nodes_coordinates
    shortest_path = nx.shortest_path(sk_graph_clean,enpoints[0],enpoints[1],weight="weight")
  File "/usr/local/lib/python3.6/dist-packages/networkx/algorithms/shortest_paths/generic.py", line 172, in shortest_path
    paths = nx.dijkstra_path(G, source, target, weight)
  File "/usr/local/lib/python3.6/dist-packages/networkx/algorithms/shortest_paths/weighted.py", line 1

In [5]:
minnie.AutoProofreadNeurons3() &

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version key by which to lookup the decomposition process version,multiplicity the total number of neurons that came from the parent segment id,cell_type_predicted morphology predicted by classifier,spine_category,n_axons Number of axon candidates identified,n_apicals Number of apicals identified,axon_length length (in um) of the classified axon skeleton,axon_area # area (in um^2) of the classified axon,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,nuclei_distance the distance to the closest nuclei (even if no matching nuclei found),n_nuclei_in_radius the number of nuclei within the search radius of 15000 belonging to that segment,n_nuclei_in_bbox the number of nuclei within the bounding box of that soma,"soma_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,axon_angle_maximum the anlge of an identified axon,spine_density_classifier the number of spines divided by skeletal length for branches analyzed in classification,n_branches_processed the number branches used for the spine density analysis,skeletal_length_processed The total skeletal length of the viable branches used for the spine density analysis,n_branches_in_search_radius the number branches existing in the search radius used for spine density,skeletal_length_in_search_radius The total skeletal length of the branches existing in the search radius used for spine density,run_time the amount of time to run (seconds)
864691134219060901,0,0.25,30.00,0,0,0,3,1,inhibitory,no_spined,0,0,0.0,0.0,73352,506.05,1,1,84299,173386,16102,11139,248,11723,24809,0,0,0,0,1,0,0,Non

In [23]:
import trimesh_utils as tu

In [25]:
tu.split_by_vertices?

# Getting the Statistics on the Initial Graph

In [ ]:
orig_presyn,orig_postsyn = (du.m65mat.NucleusSynapse() & "presyn != postsyn").fetch("presyn","postsyn")

In [ ]:
original_edges = np.vstack([orig_presyn,orig_postsyn]).T
original_edges

In [ ]:
orig_G = nx.MultiDiGraph()
orig_G.add_edges_from(original_edges)
print(len(orig_G.nodes()),len(orig_G.edges()))

G_undirected_orig = nx.Graph(orig_G)
print(len(G_undirected_orig.nodes()),len(G_undirected_orig.edges()))

In [ ]:
degree_distribution = np.array(xu.get_node_degree(orig_G,list(orig_G.nodes())))
degree_distribution_filtered = degree_distribution[degree_distribution<np.percentile(degree_distribution,99.5)]

In [ ]:
np.mean(degree_distribution_filtered),np.median(degree_distribution_filtered)

In [ ]:
print("hi")

In [ ]:
#For connected components cannot be a directed graph


# The Sizes of the components in the graph
conn_comp = list(nx.connected_components(G_undirected_orig))
conn_comp_size = [len(k) for k in conn_comp]
print(f"conn_comp_size = {conn_comp_size}")

# Pull Down and Calculating the Initial Graph

In [ ]:
synapse_table = du.proofreading_synapse_table()
synapse_table

In [ ]:
synapse_data = minnie.SynapseProofread.fetch(as_dict=True)
synapse_data_df = pd.DataFrame.from_dict(synapse_data)
synapse_data_df

In [ ]:
uniq_synapse_ids,unique_synapse_indexes,inverse_indices,synapse_id_counts = np.unique(synapse_data_df["synapse_id"].to_numpy(),
                           return_index=True,
                           return_inverse=True,
                            return_counts=True)

In [ ]:
direct_connection_synapse_idx = np.where(synapse_id_counts==2)[0]
direct_connection_synapse_mask = np.zeros(len(synapse_id_counts))
direct_connection_synapse_mask[direct_connection_synapse_idx] = 1
direct_connection_synapse_mask = direct_connection_synapse_mask.astype("bool")
row_filter = direct_connection_synapse_mask[inverse_indices]

direct_connections_df = synapse_data_df.iloc[row_filter]
direct_connections_df

In [ ]:
# Combining the table's presyns and postsyns onto the same row

df_tables = []
for synapse_type in ["presyn","postsyn"]:
    direct_connections_df_init = direct_connections_df[direct_connections_df["synapse_type"]==synapse_type]
    
    direct_connections_df_init_sorted = direct_connections_df_init.sort_values(by=['synapse_id']).reset_index(drop=True)
    
    direct_connections_df_presyn = direct_connections_df_init_sorted[[k for k in direct_connections_df_init_sorted.columns if k != "synapse_type"] ]
    
    #column_mapper = dict([(k,f"{k}_{synapse_type}") if k != "synapse_id"  else (k,k) for k in direct_connections_df_presyn.columns])
    column_mapper = dict([(k,f"{synapse_type}_{k}") for k in direct_connections_df_presyn.columns])

    presyn_df = direct_connections_df_presyn.rename(columns=column_mapper)
    df_tables.append(presyn_df)
    
direct_conn_df = pd.concat(df_tables,axis=1)
direct_conn_df

In [ ]:
edges_df = direct_conn_df[["presyn_nucleus_id","postsyn_nucleus_id"]]
connectome_edges = edges_df.to_numpy()
connectome_edges

# Preliminary Look at the Graph

In [ ]:
import networkx as nx

In [ ]:
G = nx.MultiDiGraph()
G.add_edges_from(connectome_edges)
len(G.nodes()),len(G.edges())

In [ ]:
G_undirected_non_multi = nx.Graph(G)
len(G_undirected_non_multi.nodes()),len(G_undirected_non_multi.edges())

In [ ]:
G_directed_non_multi = nx.DiGraph(G)
len(G_directed_non_multi.nodes()),len(G_directed_non_multi.edges())

In [ ]:
#For connected components cannot be a directed graph
G_undirected = G_undirected_non_multi

n_components = nx.number_connected_components(G_undirected)
print(f"Number of connected components = {n_components}")

# The Sizes of the components in the graph
conn_comp = list(nx.connected_components(G_undirected))
conn_comp_size = [len(k) for k in conn_comp]
print(f"conn_comp_size = {conn_comp_size}")

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(conn_comp_size)
ax.set_title("Number of Neurons in Connected Components")
ax.set_xlabel("Number of Neurons")
plt.show()

# Selecting the largest component

In [ ]:
G_largest = G.subgraph(conn_comp[0])
node_names_largest_comp = np.array(G_largest.nodes())
node_names_largest_comp.shape

# Plotting the Degree Distribution

In [ ]:
import graph_visualizations as gu

In [ ]:
gu.plot_degree_distribution(G,logscale=True,n_bins = 100,percentile=99,
                           font_size=20)

In [ ]:
minnie.AutoProofreadNeurons()

# Getting the statistics on the graph

In [ ]:
large_axon_nuclei = (minnie.AutoProofreadNeurons2() & "axon_length>1000").fetch("nucleus_id")
large_axon_nuclei.shape
large_axon_subgraph = G.subgraph(large_axon_nuclei)


In [ ]:
gu.plot_degree_distribution(large_axon_subgraph,logscale=True,n_bins = 100,percentile=99,
                           font_size=18,
                           title="Degree Distribution \n Neurons with Axon Length > 1000")

In [ ]:
print(f"""
Graph Statistics:

Within the Direction Connection Graph
1) {len(G.nodes())} Nodes
2) {len(G.edges())} Direct Connections
3) {len(large_axon_subgraph.nodes())} Nodes with Axon Skeletal Length > 1000
4) {len(large_axon_subgraph.edges())} Direct Connections between Nodes with Axon Skeletal Length > 1000

""")

In [ ]:
excitatory_nuclei = (minnie.AutoProofreadNeurons2() & dict(spine_category = "densely_spined")).fetch("nucleus_id")
excitatory_nuclei
G_exc = G.subgraph(excitatory_nuclei)
large_axon_nuclei_exc =  (minnie.AutoProofreadNeurons2() & dict(spine_category = "densely_spined")
                                             & "axon_length>1000").fetch("nucleus_id")
large_axon_subgraph_exc = G.subgraph(large_axon_nuclei_exc)
                              

In [ ]:
print(f"""
Graph Statistics:

* For an all densely spined excitatory graph *
Within the Direct Connection Graph
1) {len(G_exc.nodes())} Nodes
2) {len(G_exc.edges())} Direct Connections
3) {len(large_axon_subgraph_exc.nodes())} Nodes with Axon Skeletal Length > 1000
4) {len(large_axon_subgraph_exc.edges())} Direct Connections between Nodes with Axon Skeletal Length > 1000

""")

In [ ]:
"""
Steps for Decomposition:

Preprocessing:
1) Auto Glia and Nuclei Removal

Decomposition:
1) Find Soma
2) Divide into Soma and Branches
3) Skeletonize All Branches
4) Create Concept Graphs of all Limbs
5) Automatic Spine Detection, Width calculation

Proofreading:
6) Split Multi-Nuclei Cells
7) Axon and Apical Identification
8) Excitatory/Inhibitory Classification
9) E/I based mesh filtering away
10) Synapse Filtering based on mesh filtering away




"""

# Degree Distribution

In [ ]:
undirectional_graph = nx.Graph(G_largest)
no_selflooped_graph = xu.remove_selfloops(undirectional_graph)
no_selflooped_graph

In [ ]:
degree_distribution = np.array(xu.get_node_degree(no_selflooped_graph,node_names_largest_comp))
degree_distribution_filtered = degree_distribution[degree_distribution<np.percentile(degree_distribution,99.5)]

In [ ]:
print(f"Average Degree of Node: {np.mean(degree_distribution_filtered)}")
print(f"Median Degree of Node: {np.median(degree_distribution_filtered)}")

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(degree_distribution_filtered,bins=50,density=True)
ax.set_title("Degree Distribution of Neurons \n Unique Edges, No Self Loops")
ax.set_xlabel("Degree of Neuron")
ax.set_ylabel("Density")
plt.show()

# Degree Distribution In and Out

In [ ]:
np.sum(degree_distribution(G,degree_type="in")),np.sum(degree_distribution(G,degree_type="out"))

In [ ]:
np.sum(degree_distribution(G,degree_type="in_and_out"))

In [ ]:
import graph_analysis as ga
import graph_visualizations as gviz

In [ ]:
from importlib import reload
gviz = reload(gviz)

In [ ]:
in_degree = np.array(list(dict(G.in_degree()).values()))
in_degree

In [ ]:
import matplotlib.pyplot as plt
import matplotlib_utils as mu

import numpy as np

mu.histogram(in_degree,bin_max = 700,
         bin_width = 10,return_fig_ax=True,logscale=True)

In [ ]:
ga.degree_distribution(G,
                                                       degree_type="in",
                                                       percentile=95)

In [ ]:
gviz.plot_degree_distribution(G,degree_type="in",
                             percentile=100,
                              bin_width = 10,
                              bin_max = 600,
                             )

In [ ]:
gviz.plot_degree_distribution(G,degree_type="out",
                             percentile=100,
                              bin_width = 10,
                              bin_max = 600,
                             )

In [ ]:
gviz.plot_degree_distribution(G,degree_type="in",
                         bin_width=5,
                         bin_max = 800,
                        percentile=99)

In [ ]:
gviz.plot_degree_distribution(G,degree_type="out",
                         bin_width=5,
                         bin_max = 800,
                        percentile=99)

In [ ]:
gviz.plot_degree_distribution(G,degree_type="in",title="In-Degree Distribution",
                         bin_width=5,
                         bin_max = 500,
                        percentile=99)

In [ ]:
plot_degree_distribution(G,degree_type="out",title="Out-Degree Distribution",
                        percentile=99)

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(degree_distribution_filtered,bins=50,density=False)
ax.set_title("Degree Distribution of Neurons \n Unique Edges, No Self Loops")
ax.set_xlabel("Degree of Neuron")
ax.set_ylabel("Count")
ax.set_yscale("log")
plt.show()

# Getting the Soma to Soma Distance

In [ ]:
"""
Psuedoode: Pull down the Nucleus IDs and Nucleus Centers

1) Get the nucleus ids and centers
2) Adjust the centers for nm


"""

nuc_table = du.configure_nucleus_table() & "segment_id>0"
nuc_table

In [ ]:
nucleu_ids,nuc_x,nuc_y,nuc_z = nuc_table.fetch("nucleus_id","nucleus_x","nucleus_y","nucleus_z")

In [ ]:
nucleus_centers = np.vstack([nuc_x,nuc_y,nuc_z]).T * [4,4,40]
nucleus_centers

In [ ]:
nuc_id_to_center = dict([(k,v) for k,v in zip(nucleu_ids,nucleus_centers)])

In [ ]:
len(G_undirected_non_multi.edges())

In [ ]:
print("hi")

In [ ]:
import matplotlib.pyplot as plt
def graph_to_soma_to_soma_distance(G):
    total_unique_edges = np.array(list(G.edges()))
    edges_mask = np.all(total_unique_edges>0,axis=1)
    soma_distances = np.array([[nuc_id_to_center[k],nuc_id_to_center[v]] for k,v in total_unique_edges[edges_mask]])
    soma_distances_norm = np.linalg.norm(soma_distances[:,0,:] - soma_distances[:,1,:],axis=1)

    
    fig,ax = plt.subplots(1,1)
    ax.hist(soma_distances_norm/1000,bins=100,density=False)
    ax.set_title("Soma to Soma Distance of Unique Direct Connections \n After Auto Proofreading")
    ax.set_xlabel("Soma to Soma Distance (um)")
    ax.set_ylabel("Counts")
    plt.yscale("log")
    plt.xscale("linear")
    plt.show()

graph_to_soma_to_soma_distance(G)

In [ ]:
graph_to_soma_to_soma_distance(G_undirected_non_multi)

In [ ]:
total_unique_edges = np.array(list(G_undirected_non_multi.edges()))


In [ ]:
edges_mask = np.all(total_unique_edges>0,axis=1)
edges_mask

In [ ]:
soma_distances = np.array([[nuc_id_to_center[k],nuc_id_to_center[v]] for k,v in total_unique_edges[edges_mask]])
soma_distances

In [ ]:
soma_distances.shape

In [ ]:
soma_distances_norm = np.linalg.norm(soma_distances[:,0,:] - soma_distances[:,1,:],axis=1)
soma_distances_norm.shape

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1)
ax.hist(soma_distances_norm/1000,bins=100,density=False)
ax.set_title("Soma to Soma Distance of Unique Direct Connections \n After Auto Proofreading")
ax.set_xlabel("Soma to Soma Distance (um)")
ax.set_ylabel("Counts")
plt.yscale("log")
plt.xscale("linear")
plt.show()

In [ ]:
alpha_level = 0.7

import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1)
ax.hist(total_distances_no_self_loops/1000,bins=100,density=False,color="blue",label="Before Auto Proofreading",alpha=alpha_level)
ax.hist(total_distances_no_self_loops_auto/1000,bins=100,density=False,color="red",label="After Auto Proofreading",alpha=alpha_level)
ax.set_title("Soma to Soma Distance of Direct Connections")
ax.set_xlabel("Soma to Soma Distance (um)")
ax.set_ylabel("Counts")

ax.set_yscale("log")
ax.set_xscale("linear")
ax.legend()
plt.show()